# MVP Pipeline de Dados – Recursos Humanos  
## Análise de Retenção e Eficiência Organizacional  

**Vanessa Araújo**  

PUC-RJ – MBA em Ciência de Dados e Analytics  
Disciplina de Engenharia de Dados  

---

###Script ETL para carga na camada SILVER

###Imports
Imports das bibliotecas necessárias para o funcionamento do script.

In [0]:
import uuid
import warnings
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
from pyspark.sql.functions import col, to_date, trim
from pyspark.sql.functions import col, trim, lower, monotonically_increasing_id
from pyspark.sql.functions import when, trim

###Carga de Dados

Os dados serão carregados a partir da camada bronze para os tratamentos necessários.

In [0]:
spark = SparkSession.builder.getOrCreate()

df_bronze = spark.table("bronze.rhg_dataset")
df_bronze.printSchema()

root
 |-- nome_funcionario: string (nullable = true)
 |-- id_funcionario: integer (nullable = true)
 |-- id_casado: long (nullable = true)
 |-- id_status_conjulgal: long (nullable = true)
 |-- id_genero: long (nullable = true)
 |-- id_status_funcionario: long (nullable = true)
 |-- id_departamento: long (nullable = true)
 |-- id_pontuacao_desempenho: long (nullable = true)
 |-- id_recrutamento_diversidade: long (nullable = true)
 |-- salario: float (nullable = true)
 |-- id_desligado: long (nullable = true)
 |-- id_cargo: long (nullable = true)
 |-- cargo: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- cep: long (nullable = true)
 |-- aniversario: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- estado_civil: string (nullable = true)
 |-- cidadania: string (nullable = true)
 |-- latino: string (nullable = true)
 |-- raca: string (nullable = true)
 |-- data_contratacao: date (nullable = true)
 |-- data_rescisao: date (nullable = true)
 |-- motivo_re

###Análise Exploratória Inicial dos Dados
Antes da aplicação de qualquer transformação, foi realizada uma análise exploratória para avaliação da qualidade dos dados.


In [0]:
%sql
SELECT *
FROM bronze.rhg_dataset
LIMIT 10;

nome_funcionario,id_funcionario,id_casado,id_status_conjulgal,id_genero,id_status_funcionario,id_departamento,id_pontuacao_desempenho,id_recrutamento_diversidade,salario,id_desligado,id_cargo,cargo,estado,cep,aniversario,sexo,estado_civil,cidadania,latino,raca,data_contratacao,data_rescisao,motivo_rescisao,situacao_emprego,departamento,nome_gestor,id_gestor,fonte_recrutamento,avaliacao_desempenho,pesquisa_engajamento,satisfacao_funcionario,contagem_projetos_especiais,data_ultimo_feedback,dias_atrasado_ultimo_30,ausencias
"Adinolfi, Wilson K",10026,0,0,1,1,5,4,0,62506.0,0,19,Técnico de Produção I,MA,1960,07/10/83,H,Solteiro,Cidadão americano,Não,Branco,2011-07-05,null,N/A - Ainda empregado,Ativo,Produção,Michael Albert,22.0,LinkedIn,Excede,460,Muito Satisfeito,0,2019-01-17,0,1
"Ait Sidi, Karthikeyan",10084,1,1,1,5,3,3,0,104437.0,1,27,Sr. DBA,MA,2148,05/05/75,H,Casado,Cidadão americano,Não,Branco,2015-03-30,2016-06-16,Mudança de carreira,Rescindido voluntariamente,T.I,Simon Roup,4.0,Indeed,Atende totalmente,496,Neutro,6,2016-02-24,0,17
"Akinkuolie, Sarah",10196,1,1,0,5,5,3,0,64955.0,1,20,Técnico de Produção II,MA,1810,09/19/88,M,Casado,Cidadão americano,Não,Branco,2011-07-05,2012-09-24,Horas,Rescindido voluntariamente,Produção,Kissy Sullivan,20.0,LinkedIn,Atende totalmente,302,Neutro,0,2012-05-15,0,3
"Alagbe,Trina",10088,1,1,0,1,5,3,0,64991.0,0,19,Técnico de Produção I,MA,1886,09/27/88,M,Casado,Cidadão americano,Não,Branco,2008-01-07,null,N/A - Ainda empregado,Ativo,Produção,Elijiah Gray,16.0,Indeed,Atende totalmente,484,Muito Satisfeito,0,2019-01-03,0,15
"Anderson, Carol",10069,0,2,0,5,5,3,0,50825.0,1,19,Técnico de Produção I,MA,2169,09/08/89,M,Divorciado,Cidadão americano,Não,Branco,2011-07-11,2016-09-06,Retorno à escola,Rescindido voluntariamente,Produção,Webster Butler,39.0,Pesquisa Google,Atende totalmente,500,Satisfeito,0,2016-02-01,0,2
"Anderson, Linda",10002,0,0,0,1,5,4,0,57568.0,0,19,Técnico de Produção I,MA,1844,05/22/77,M,Solteiro,Cidadão americano,Não,Branco,2012-01-09,null,N/A - Ainda empregado,Ativo,Produção,Amy Dunn,11.0,LinkedIn,Excede,500,Muito Satisfeito,0,2019-01-07,0,15
"Andreola, Colby",10194,0,0,0,1,4,3,0,95660.0,0,24,Engenheiro de software,MA,2110,05/24/79,M,Solteiro,Cidadão americano,Não,Branco,2014-11-10,null,N/A - Ainda empregado,Ativo,Engenharia de Software,Alex Sweetwater,10.0,LinkedIn,Atende totalmente,304,Neutro,4,2019-01-02,0,19
"Athwal, Sam",10062,0,4,1,1,5,3,0,59365.0,0,19,Técnico de Produção I,MA,2199,02/18/83,H,Viúva,Cidadão americano,Não,Branco,2013-09-30,null,N/A - Ainda empregado,Ativo,Produção,Ketsia Liebig,19.0,Indicação de funcionários,Atende totalmente,500,Satisfeito,0,2019-02-25,0,19
"Bachiochi, Linda",10114,0,0,0,3,5,3,1,47837.0,0,19,Técnico de Produção I,MA,1902,02/11/70,M,Solteiro,Cidadão americano,Não,Negro ou afro-americano,2009-07-06,null,N/A - Ainda empregado,Ativo,Produção,Brannon Miller,12.0,Feira de Empregos de Diversidade,Atende totalmente,446,Neutro,0,2019-01-25,0,4
"Bacong, Alejandro",10250,0,2,1,1,3,3,0,50178.0,0,14,Suporte de TI,MA,1886,01/07/88,H,Divorciado,Cidadão americano,Não,Branco,2015-01-05,null,N/A - Ainda empregado,Ativo,T.I,Peter Monroe,7.0,Indeed,Atende totalmente,500,Muito Satisfeito,6,2019-02-18,0,16


**3.2 Quantidade total de registros**

In [0]:
%sql
SELECT COUNT(*) AS total_registros
FROM bronze.rhg_dataset;

total_registros
1555


**3.3 Verificação de valores nulos por coluna**

In [0]:
%sql
SELECT
  COUNT(*) - COUNT(id_funcionario)              AS null_id_funcionario,
  COUNT(*) - COUNT(departamento)                AS null_departamento,
  COUNT(*) - COUNT(cargo)                       AS null_cargo,
  COUNT(*) - COUNT(salario)                     AS null_salario,
  COUNT(*) - COUNT(id_desligado)                AS null_id_desligado,
  COUNT(*) - COUNT(avaliacao_desempenho)        AS null_avaliacao_desempenho,
  COUNT(*) - COUNT(pesquisa_engajamento)        AS null_pesquisa_engajamento,
  COUNT(*) - COUNT(ausencias)                   AS null_ausencias
FROM bronze.rhg_dataset;

null_id_funcionario,null_departamento,null_cargo,null_salario,null_id_desligado,null_avaliacao_desempenho,null_pesquisa_engajamento,null_ausencias
0,0,0,0,0,0,0,0


Não foram identificados registros duplicados para a chave funcional emp_id, indicando consistência na identificação dos colaboradores.

**3.4 Verificação de duplicidade de colaboradores**

In [0]:
%sql
SELECT
  id_funcionario,
  COUNT(*) AS qtd
FROM bronze.rhg_dataset
GROUP BY id_funcionario
HAVING COUNT(*) > 1;

id_funcionario,qtd
10203,5
10255,5
10288,5
10082,5
10257,5
10187,5
10268,5
10245,5
10214,5
10001,5


A análise de duplicidade baseada na chave funcional id_funcionario identificou 311 colaboradores com mais de um registro associado. Essa característica indica que o conjunto de dados possui natureza histórica, contendo múltiplos registros para um mesmo colaborador ao longo do tempo, possivelmente relacionados a avaliações de desempenho, pesquisas de engajamento ou atualizações cadastrais.

###Tratamentos Aplicados na Camada Silver

Considerando a boa qualidade dos dados, os tratamentos aplicados concentraram-se em padronização de tipos de dados e consistência semântica, sem exclusão de registros.

###Padronização de tipos e campos

In [0]:
df_silver = (
    df_bronze
    .withColumn("id_funcionario", col("id_funcionario").cast("string"))
    .withColumn("id_departamento", col("id_departamento").cast("string"))
    .withColumn("id_cargo", col("id_cargo").cast("string"))
    .withColumn("id_gestor", col("id_gestor").cast("string"))
    .withColumn("salario", col("salario").cast("double"))
    .withColumn("ausencias", col("ausencias").cast("int"))
    .withColumn("dias_atrasado_ultimo_30", col("dias_atrasado_ultimo_30").cast("int"))
    .withColumn("contagem_projetos_especiais", col("contagem_projetos_especiais").cast("int"))
)

###Padronização semântica de campos categóricos
Campos categóricos passaram por remoção de espaços extras para evitar inconsistências analíticas.

In [0]:
from pyspark.sql.functions import trim
df_silver = (

    df_silver
    .withColumn("avaliacao_desempenho", trim(col("avaliacao_desempenho")))
    .withColumn("satisfacao_funcionario", trim(col("satisfacao_funcionario")))
    .withColumn("departamento", trim(col("departamento")))
    .withColumn("cargo", trim(col("cargo")))
)

###Conversão de indicadores lógicos

O campo id_desligado foi mantido como indicador binário para análises de rotatividade.

In [0]:
df_silver = df_silver.withColumn(
    "desligado_flag",
    col("id_desligado").cast("int")
)

In [0]:
from pyspark.sql.functions import when, trim, col
df_fact_employee = df_bronze.withColumn(

    "employee_satisfaction",
    when(trim(col("satisfacao_funcionario")) == "Muito Insatisfeito", 1)
    .when(trim(col("satisfacao_funcionario")) == "Insatisfeito", 2)
    .when(trim(col("satisfacao_funcionario")) == "Neutro", 3)
    .when(trim(col("satisfacao_funcionario")) == "Satisfeito", 4)
    .when(trim(col("satisfacao_funcionario")) == "Muito Satisfeito", 5)
    .otherwise(None)
)

###4.2 Base tratada para modelagem Silver
Após os tratamentos, os dados permaneceram em um DataFrame intermediário (df_silver), que serviu como base para a separação dos dados em tabelas dimensionais e fato na camada Silver, preservando o histórico completo dos colaboradores.

In [0]:
df_silver.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("silver.rh_employees")

###5 Separação dos Dados em Tabelas – Camada Silver
Após a etapa de análise e tratamento dos dados provenientes da camada Bronze, os dados foram organizados segundo um modelo dimensional, com separação entre tabelas dimensão e tabela fato, visando facilitar consultas analíticas e garantir integridade semântica na camada Silver.

**5.1 Dimensão Funcionário – silver.dim_employee**

Tabela dimensão que armazena informações cadastrais e demográficas dos colaboradores.

In [0]:
from pyspark.sql.functions import col, trim

df_dim_employee = (
    spark.table("bronze.rhg_dataset")
    .select(
        col("id_funcionario").cast("string").alias("employee_id"),
        trim(col("nome_funcionario")).alias("employee_name"),
        trim(col("sexo")).alias("gender"),
        trim(col("raca")).alias("race"),
        trim(col("estado_civil")).alias("marital_status"),
        trim(col("cidadania")).alias("citizenship"),
        trim(col("latino")).alias("hispanic_latino")
    )
    .dropDuplicates(["employee_id"])
)

# Grava a dimensão na camada Silver garantindo alinhamento de schema
df_dim_employee.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("silver.dim_employee")


**5.2 Dimensão Departamento – silver.dim_department**

In [0]:
from pyspark.sql.functions import col, trim

# Criação da Dimensão Departamento
df_dim_department = (
    df_silver
    .select(
        col("id_departamento").cast("string").alias("department_id"),
        trim(col("departamento")).alias("department_name")
    )
    .dropna(subset=["department_id"])
    .dropDuplicates(["department_id"])
)

# Persistência na camada Silver
df_dim_department.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver.dim_department")

print("Dimensão Departamento (silver.dim_department) carregada com sucesso.")

Dimensão Departamento (silver.dim_department) carregada com sucesso.


**5.3 Dimensão Cargo (Position)**

In [0]:
from pyspark.sql.functions import col, trim

# Criação da Dimensão Cargo
df_dim_position = (
    df_silver
    .select(
        col("id_cargo").cast("string").alias("position_id"),
        trim(col("cargo")).alias("position_name")
    )
    .dropna(subset=["position_id"])
    .dropDuplicates(["position_id"])
)

# Persistência na camada Silver
df_dim_position.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver.dim_position")

print("Dimensão Cargo (silver.dim_position) carregada com sucesso.")


Dimensão Cargo (silver.dim_position) carregada com sucesso.


**5.4 Dimensão Gestor**

In [0]:
from pyspark.sql.functions import col, trim

# Criação da Dimensão Gestor
df_dim_manager = (
    df_silver
    .select(
        col("id_gestor").cast("string").alias("manager_id"),
        trim(col("nome_gestor")).alias("manager_name")
    )
    .dropna(subset=["manager_id"])
    .dropDuplicates(["manager_id"])
)

# Persistência na camada Silver
df_dim_manager.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver.dim_manager")

print("Dimensão Gestor (silver.dim_manager) carregada com sucesso.")


Dimensão Gestor (silver.dim_manager) carregada com sucesso.


**5.5 Tabela Fato – fact_employee**

In [0]:
from pyspark.sql.functions import col, trim

# Criação da Tabela Fato Funcionário
df_fact_employee = (
    df_silver
    .select(
        col("id_funcionario").cast("string").alias("employee_id"),
        col("id_departamento").cast("string").alias("department_id"),
        col("id_cargo").cast("string").alias("position_id"),
        col("id_gestor").cast("string").alias("manager_id"),
        col("data_contratacao").alias("hire_date"),
        col("data_rescisao").alias("termination_date"),
        col("salario").cast("double").alias("salary"),
        col("ausencias").cast("int").alias("absences"),
        col("dias_atrasado_ultimo_30").cast("int").alias("days_late_last_30"),
        trim(col("avaliacao_desempenho")).alias("performance_score"),
        col("pesquisa_engajamento").cast("int").alias("employee_satisfaction"),
        col("contagem_projetos_especiais").cast("int").alias("special_projects_count")
    )
)

# Persistência na camada Silver
df_fact_employee.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver.fact_employee")

print("Tabela Fato Funcionário (silver.fact_employee) carregada com sucesso.")


Tabela Fato Funcionário (silver.fact_employee) carregada com sucesso.


###Teste de carga da Tabela
Teste de carga da tabela, para garantir o sucesso da operação.

In [0]:
%sql
SELECT *
FROM silver.dim_employee
LIMIT 10

employee_id,employee_name,gender,race,marital_status,citizenship,hispanic_latino
10096,"Lundy, Susan",M,Branco,Viúva,Cidadão americano,Não
10283,"Gilles, Alex",H,Negro ou afro-americano,Casado,Cidadão americano,Não
10311,"Dee, Randy",H,Branco,Casado,Cidadão americano,Não
10266,"Stoica, Rick",H,Asiático,Casado,Cidadão americano,Não
10101,"Gonzalez, Maria",M,Branco,Separado,Cidadão americano,Sim
10126,"Saada, Adell",M,Branco,Casado,Cidadão americano,Não
10305,"Forrest, Alex",H,Branco,Casado,Cidadão americano,Não
10280,"Estremera, Miguel",H,Branco,Solteiro,Cidadão americano,Não
10104,"Nowlan, Kristie",M,Branco,Solteiro,Cidadão americano,Não
10308,"Fernandes, Nilson",H,Branco,Casado,Cidadão americano,Não


In [0]:
%sql
SELECT *
FROM silver.dim_department
LIMIT 10

department_id,department_name
2,Escritório Executivo
6,Venda
4,Engenharia de Software
5,Produção
1,Escritórios administrativos
3,T.I


In [0]:
%sql
SELECT *
FROM silver.dim_position
LIMIT 10

position_id,position_name
2,Assistente Administrativo
14,Suporte de TI
9,Analista de Dados
28,Engenheiro de rede sênior
16,Presidente e CEO
6,Diretor de Tecnologia
22,Desenvolvedor de BI Sênior
29,Arquiteto de Dados Principal
12,Diretor de TI
7,Arquiteto de Dados


In [0]:
%sql
SELECT *
FROM silver.dim_manager
LIMIT 10

manager_id,manager_name
7.0,Peter Monroe
13.0,Brian Champaigne
16.0,Elijiah Gray
10.0,Alex Sweetwater
4.0,Simon Roup
11.0,Amy Dunn
3.0,Brandon R. LeBlanc
14.0,David Stanley
39.0,Webster Butler
6.0,Eric Dougall


In [0]:
%sql
SELECT *
FROM silver.fact_employee
LIMIT 10

employee_id,department_id,position_id,manager_id,hire_date,termination_date,salary,absences,days_late_last_30,performance_score,employee_satisfaction,special_projects_count
10026,5,19,22.0,2011-07-05,null,62506.0,1,0,Excede,460,0
10084,3,27,4.0,2015-03-30,2016-06-16,104437.0,17,0,Atende totalmente,496,6
10196,5,20,20.0,2011-07-05,2012-09-24,64955.0,3,0,Atende totalmente,302,0
10088,5,19,16.0,2008-01-07,null,64991.0,15,0,Atende totalmente,484,0
10069,5,19,39.0,2011-07-11,2016-09-06,50825.0,2,0,Atende totalmente,500,0
10002,5,19,11.0,2012-01-09,null,57568.0,15,0,Excede,500,0
10194,4,24,10.0,2014-11-10,null,95660.0,19,0,Atende totalmente,304,4
10062,5,19,19.0,2013-09-30,null,59365.0,19,0,Atende totalmente,500,0
10114,5,19,12.0,2009-07-06,null,47837.0,4,0,Atende totalmente,446,0
10250,3,14,7.0,2015-01-05,null,50178.0,16,0,Atende totalmente,500,6
